# テストデータの欠損値を代表値で補完する

In [1]:
import os
os.chdir('../../')

In [6]:
import numpy as np
import pandas as pd
from scr.util import *

In [40]:
df_test = pd.read_csv('data/test.csv')

In [41]:
df_test.isna().sum()

id                          0
Age                        93
TypeofContact              12
CityTier                    0
DurationOfPitch           131
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups          24
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              42
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              49
customer_info               0
dtype: int64

## 正規化 + 欠損値補完 (意味がある特徴量)

In [42]:
#df_train['Age'] = df_train['Age'].astype(str).apply(japanese_to_int)
df_test['Age'] = df_test['Age'].astype(str).apply(japanese_to_int)

#df_train['TypeofContact'] = df_train['TypeofContact'].fillna('No')
df_test['TypeofContact'] = df_test['TypeofContact'].fillna('No')

#df_train['DurationOfPitch'] = df_train['DurationOfPitch'].dropna().apply(convert_to_second)
df_test['DurationOfPitch'] = df_test['DurationOfPitch'].dropna().apply(convert_to_second)

#df_train['Gender'] = df_train['Gender'].apply(normalize_gender)
df_test['Gender'] = df_test['Gender'].apply(normalize_gender)

#df_train['NumberOfFollowups'] = df_train['NumberOfFollowups'].apply(lambda x: x if x < 100 else x / 100)
df_test['NumberOfFollowups'] = df_test['NumberOfFollowups'].apply(lambda x: x if x < 100 else x / 100)
#df_train['NumberOfFollowups'] = df_train['NumberOfFollowups'].fillna(0)
df_test['NumberOfFollowups'] = df_test['NumberOfFollowups'].fillna(0)

#df_train['ProductPitched'] = df_train['ProductPitched'].apply(normalize_product_pitched_1)
#df_train['ProductPitched'] = df_train['ProductPitched'].apply(normalize_product_pitched_2)
df_test['ProductPitched'] = df_test['ProductPitched'].apply(normalize_product_pitched_1)
df_test['ProductPitched'] = df_test['ProductPitched'].apply(normalize_product_pitched_2)

#df_train['NumberOfTrips'] = df_train['NumberOfTrips'].apply(normalize_trips)
df_test['NumberOfTrips'] = df_test['NumberOfTrips'].apply(normalize_trips)

#df_train['Designation'] = df_train['Designation'].apply(normalize_designation_1)
#df_train['Designation'] = df_train['Designation'].apply(normalize_designation_2)
df_test['Designation'] = df_test['Designation'].apply(normalize_designation_1)
df_test['Designation'] = df_test['Designation'].apply(normalize_designation_2)

#df_train['MonthlyIncome'] = df_train['MonthlyIncome'].apply(normalize_monthly_income)
df_test['MonthlyIncome'] = df_test['MonthlyIncome'].apply(normalize_monthly_income)

#df_train[['Marry', 'Car', 'Child']] = df_train['customer_info'].apply(divide_customer_info)
df_test[['Marry', 'Car', 'Child']] = df_test['customer_info'].apply(divide_customer_info)
#df_train['Marry'] = df_train['Marry'].apply(normalize_info_1)
df_test['Marry'] = df_test['Marry'].apply(normalize_info_1)
#df_train['Car'] = df_train['Car'].apply(normalize_info_2)
df_test['Car'] = df_test['Car'].apply(normalize_info_2)
#df_train['Child'] = df_train['Child'].apply(normalize_info_3)
df_test['Child'] = df_test['Child'].apply(normalize_info_3)

In [43]:
drop_list = ['customer_info']

#df_train = df_train.drop(columns=drop_list)
df_test = df_test.drop(columns=drop_list)

## 中央値で補完する

In [44]:
tmp = df_test.loc[df_test['Age'] != 0, 'Age']
df_test.loc[df_test['Age'] == 0, 'Age'] = tmp.median()

In [45]:
tmp = df_test.loc[~(df_test['DurationOfPitch'].isna()), 'DurationOfPitch']
df_test.loc[df_test['DurationOfPitch'].isna(), 'DurationOfPitch'] = tmp.median()

In [46]:
tmp = df_test.loc[~(df_test['NumberOfTrips'] == 'nan'), 'NumberOfTrips']
df_test.loc[df_test['NumberOfTrips'] == 'nan', 'NumberOfTrips'] = tmp.median()

In [47]:
tmp = df_test.loc[~(df_test['MonthlyIncome'].isna()), 'MonthlyIncome']
df_test.loc[df_test['MonthlyIncome'].isna(), 'MonthlyIncome'] = tmp.median()

# 数値変換 (DurationOfPitch, MonthlyIncome)

In [48]:
df_test['DurationOfPitch'] = np.log1p(df_test['DurationOfPitch'])
df_test['MonthlyIncome'] = np.log1p(df_test['MonthlyIncome'])

## 年代特徴量作成

In [49]:
def age_to_agegroup(age):
    if age == 0:
        return np.nan    
    elif age < 20:
        return "10s"
    elif age < 30:
        return "20s"
    elif age < 40:
        return "30s"
    elif age < 50:
        return "40s"
    elif age < 60:
        return "50s"
    else:
        return "60s"

# df_train['AgeGroup'] = df_train['Age'].apply(age_to_agegroup)
df_test['AgeGroup'] = df_test['Age'].apply(age_to_agegroup)

# CSV出力

In [50]:
df_test.to_csv('data/feature_engineered/null_del_xxx/test_null_median.csv', index=False)